In [ ]:
%pip install -qU openai
%pip install -qU langchain-openai
%pip install -qU langchain-community
%pip install -qU langchain-qdrant
%pip install -qU rouge-score

api keys available at https://platform.openai.com/settings/organization/api-keys

In [10]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

openai_key = os.getenv("OPENAI_API_KEY")
openai_organization = os.getenv("OPENAI_ORGANIZATION")
openai_project_id = os.getenv("OPENAI_PROJECT_ID")


In [ ]:
from openai import OpenAI

model = "gpt-4o-mini"

openaiclient = OpenAI(
  organization=openai_organization,
  project=openai_project_id,
  api_key=openai_key
)

In [ ]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client import models, QdrantClient
from langchain_core.vectorstores import VectorStoreRetriever

qdrant = QdrantClient(path="./qdrant_data")

#embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = SentenceTransformerEmbeddings(model_name="paraphrase-MiniLM-L6-v2")



In [19]:
from sentence_transformers import SentenceTransformer
embeddingModel = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [21]:
searchedText = """Please explain to me rules of Battle in Almoravid"""

In [12]:
def debug_hit (score, payload, filename):
    with open(filename, "w") as file:
        file.write("Score: " + str(score) + "\n")
        file.write("Payload: " + str(payload) + "\n")

In [16]:
[(index, value) for index, value in enumerate(["a", "b", "c"])]

[(0, 'a'), (1, 'b'), (2, 'c')]

In [23]:
hits = qdrant.search(
    collection_name="demo_cosine",
    query_vector=embeddingModel.encode(searchedText).tolist(),
    limit=5,
)

for index, hit in enumerate(hits):
    print(hit)
    print("score:", hit.score, "*****TXT******", hit.payload["source"][:80])
    debug_hit(hit.score, hit.payload, f"debug/output{index}.txt")


id=40 version=0 score=0.5186433699364841 payload={'source': '6.3.4 Plowing. At the end of the second 40 Days of Winter (box \n8), each Lord at a Siege (only) reduces his Carts and Mules each \nto ½ their number (rounded up).\n6.3.5 Arts of War. In the first Spring Levy (box 9), \ndraw Capabilities instead of Events (3.1).\n© 2021 GMT Games, LLC\nAlmoravid\n35\nKey Terms\n40 Days—Calendar box, a turn (1.3.2). \nActive—Acting on Campaign (4.0).\nAdjacent—Linked by a Way (1.3.1). \nAfrican Foot and Horse—Forces types (1.6). \nAllegiance—Friendly, Neutral, Enemy (1.3.1).\nApproach—March to enemy (4.3.4). \nArmor—Unit Protection roll (4.4.2). \nArray—Lords in Battle or Storm (4.4.1). \nArts of War—card, deck (1.9.1, 3.1). \nAsset—Coin, Loot, Provender, Transport (1.7). \nAttack—Engage in Battle or Storm as the Active side (4.4, 4.5.2). \nAutumn—a Season (2.2.1). \nAvoid Battle—Back off from Approach (4.3.4). \nBattle—Fight due to March (4.4). \nBattle mat—Array option (4.4.1). \nBesieged—Un

In [25]:

from langchain_openai.chat_models import ChatOpenAI

# LLM
llm = ChatOpenAI()

In [27]:
answer =  openaiclient.chat.completions.create(
        model = model,
        messages= [
            {"role": "system", "content": "You are an expert player of Levy & Campaign games, particularly Almoravid. You are helpful and always willing to explain the rules and strategies of the game" },
            {"role": "system", "content": 
                hits[0].payload['source'] },            
            {"role": "user", "content": "Please explain to me rules of Battle in Almoravid"}
        ]
    )

In [32]:
answer.choices

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Battle in Almoravid game is a major aspect that involves engaging with an enemy through direct confrontation. Here's a breakdown of the rules governing battles:\n\n### 1. **Initiating Battle**\n- A battle occurs when an active Lord (the player’s representative on the game board) marches into an enemy's linked territory or adjacent stronghold while performing an *Approach*.\n  \n### 2. **Arraying Forces**\n- Before a battle begins, players must *Array* their Lords. This means placing their units (Foot or Horse) in a specified formation on the battle mat. Arraying includes determining which units will fight and in what order they will be engaged.\n\n### 3. **Rolling for Combat**\n- Combat is resolved using a dice roll for each participating unit during the battle:\n  - **Attack Rolls:** The attacker rolls dice, aiming to exceed a target number based on the enemy's unit defenses.\n  - **Defense Ro

In [29]:
answer.choices[0].message.content

"Battle in Almoravid game is a major aspect that involves engaging with an enemy through direct confrontation. Here's a breakdown of the rules governing battles:\n\n### 1. **Initiating Battle**\n- A battle occurs when an active Lord (the player’s representative on the game board) marches into an enemy's linked territory or adjacent stronghold while performing an *Approach*.\n  \n### 2. **Arraying Forces**\n- Before a battle begins, players must *Array* their Lords. This means placing their units (Foot or Horse) in a specified formation on the battle mat. Arraying includes determining which units will fight and in what order they will be engaged.\n\n### 3. **Rolling for Combat**\n- Combat is resolved using a dice roll for each participating unit during the battle:\n  - **Attack Rolls:** The attacker rolls dice, aiming to exceed a target number based on the enemy's unit defenses.\n  - **Defense Rolls:** The defender also rolls to protect their units from harm.\n\n### 4. **Types of Units*

In [ ]:
answer =  openaiclient.chat.completions.create(
        model = model,
        messages= [
            {"role": "system", "content": "You are an expert player of Levy & Campaign games, particularly Almoravid. You are helpful and always willing to explain the rules and strategies of the game" },
            {"role": "system", "content": 
                """
NOTE: Battles last at least one Round.\n• This Round, \nthe Enemy gains a Pursuit advantage against the\nConceding side’s Hits (see below).\n• \n Place the Pursuit marker between the \nsides, pointing from Pursuing to Con-\nceding side.\nREPOSITION: Then, in each Round after the first, both sides \nreposition Lords as follows (in this order).\n• Rout. Remove \nRouted Lords from the Battle Array. They no\nlonger occupy any Array position nor participate further in the\nBattle until Ending the Battle (4.4.3).\n• Advance. \nSlide any Unrouted Lords in Reserve into any empty\nFront positions (one each).\n• Center.\n If a Front center position remains empty, first the At-\ntacker then the Defender must select and slide one of that side’s \nLords from either left or right front to fill its empty Front-cen-\nter position.\nSTRIKE: The Forces of Lords in Front Strike the Forces of the \nLord directly opposite them or—if Flanking—those of the closest \nFront Enemy Lord. \n• Flanking. Whenever a Lord at Front has no Enemy Lord\ndirectly opposite, he Strikes the closest Front Enemy Lord; \ncenter may choose left or right. Total all Hits from Flank-\ning Lords plus the directly opposed Lord, then round up. A \nFlanking Lord may absorb Hits from a Flanked Lord, at the \nowner’\ns option. \n• Initiative.  Strike step by step in the order shown below; each\nletter designates \na separate step. Within each step, Striking\nLords choose the order of Strike, Lord by Lord (or Lord with\nFlanking Lord[s]). Then any Flanking Lords choose wheth-\ner to absorb Strikes before opposed Lords. Resolve all Hits,\nProtection rolls, and Rout before Striking in the next step, not\nsimultaneously. EXAMPLE: Attackers take Hits from Defend-\ning Missiles before Attacking Missiles Strike.\n1. Missile steps\na. Defending Missiles\nb. Attacking Missiles \n2. Melee steps\na. Defending Horse\nb. Attacking Horse \nc. Defending Foot\nd. Attacking Foot \n•\n \uf075 Javelins: A Lord with Javelins may use them in\nany one Round of that side’s choice only, an-\nnouncing so at the moment of use and flipping the\nJavelins marker on his mat to the red “X”.\nA portion of the Forces table on the foldout play aid.\nTOTAL HITS: Each unit causes ½, 1, or 2 Hits, depending on \nForces and Strike type—see the Forces table. Total all Hits for \nthat step, including Flanking plus directly opposing Lords, round-\ning up. If separate Capabilities give a unit “Missiles x½” and \n“Missiles x1”, it has Missiles x1; if “Missiles x1” and target Ar-\nmor -1, it has both. NOTE: You do not roll to generate Hits – they \ngenerate automatically based on the Forces Striking.\n• Mixed Missiles: \nWhen Crossbows combine with other Mis-\nsiles, round Hits in favor of Crossbows. EXAMPLE: Three \nCrossbow Men-at-Arms and three Militia Bowmen would yield \na total of three Hits, of which two would count as Crossbows.\n• \n Pursuit. The Conceding side halves its \ntotal Hits against the Pursuing side. \nRound all fractions up by step.\n© 2021 GMT Games, LLC\nAlmoravid\n19\nAPPLY HITS TO LORDS: Hits apply to the Forces of the op-\nposed, Flanked, or Flanking Enemy Lord. A Player with a Flank-\ning Lord selects either the Flanking or directly opposed Lord to \ntake Hits. Whenever a Lord Routs to create a new Flanking situa-\ntion, apply remaining Hits accordingly. \nPROTECTION: Strongholds, Siegeworks, cards, and most \nForces types give saving rolls that can nullify Hits—Walls, Ar-\nmor, Evade, and Unarmored Protection. Crossbows and cards can \nalter the rolls needed. See the Forces and Strongholds tables.\nROLL WALLS: If this is a Storm or Sally (4.5.2-.3), a Lord ben-\nefiting from Walls or Siegeworks first rolls dice equal to the total \nnumber of Hits just received in that step before assigning Hits to \nunits. Each roll within the Walls range specified (often, 1-4; 1-3 \nwith Siege Towers) or less than or equal to the total Siege mark-\ners there cancels one Hit.'} vector=None shard_key=None order_value=None
score: 0.49804240606087363 *****TXT****** NOTE: Battles last at least one Round.
• This Round, 
the Enemy gains a Pursuit advantage against the
Conceding side’s Hits (see below).
• 
 Place the Pursuit marker between the 
sides, pointing from Pursuing to Con-
ceding side.
REPOSITION: Then, in each Round after the first, both sides 
reposition Lords as follows (in this order).
• Rout. Remove 
Routed Lords from the Battle Array. They no
longer occupy any Array position nor participate further in the
Battle until Ending the Battle (4.4.3).
• Advance. 
Slide any Unrouted Lords in Reserve into any empty
Front positions (one each).
• Center.
 If a Front center position remains empty, first the At-
tacker then the Defender must select and slide one of that side’s 
Lords from either left or right front to fill its empty Front-cen-
ter position.
STRIKE: The Forces of Lords in Front Strike the Forces of the 
Lord directly opposite them or—if Flanking—those of the closest 
Front Enemy Lord. 
• Flanking. Whenever a Lord at Front has no Enemy Lord
directly opposite, he Strikes the closest Front Enemy Lord; 
center may choose left or right. Total all Hits from Flank-
ing Lords plus the directly opposed Lord, then round up. A 
Flanking Lord may absorb Hits from a Flanked Lord, at the 
owner’
s option. 
• Initiative.  Strike step by step in the order shown below; each
letter designates 
a separate step. Within each step, Striking
Lords choose the order of Strike, Lord by Lord (or Lord with
Flanking Lord[s]). Then any Flanking Lords choose wheth-
er to absorb Strikes before opposed Lords. Resolve all Hits,
Protection rolls, and Rout before Striking in the next step, not
simultaneously. EXAMPLE: Attackers take Hits from Defend-
ing Missiles before Attacking Missiles Strike.
1. Missile steps
a. Defending Missiles
b. Attacking Missiles 
2. Melee steps
a. Defending Horse
b. Attacking Horse 
c. Defending Foot
d. Attacking Foot 
•
  Javelins: A Lord with Javelins may use them in
any one Round of that side’s choice only, an-
nouncing so at the moment of use and flipping the
Javelins marker on his mat to the red “X”.
A portion of the Forces table on the foldout play aid.
TOTAL HITS: Each unit causes ½, 1, or 2 Hits, depending on 
Forces and Strike type—see the Forces table. Total all Hits for 
that step, including Flanking plus directly opposing Lords, round-
ing up. If separate Capabilities give a unit “Missiles x½” and 
“Missiles x1”, it has Missiles x1; if “Missiles x1” and target Ar-
mor -1, it has both. NOTE: You do not roll to generate Hits – they 
generate automatically based on the Forces Striking.
• Mixed Missiles: 
When Crossbows combine with other Mis-
siles, round Hits in favor of Crossbows. EXAMPLE: Three 
Crossbow Men-at-Arms and three Militia Bowmen would yield 
a total of three Hits, of which two would count as Crossbows.
• 
 Pursuit. The Conceding side halves its 
total Hits against the Pursuing side. 
Round all fractions up by step.
© 2021 GMT Games, LLC
Almoravid
19
APPLY HITS TO LORDS: Hits apply to the Forces of the op-
posed, Flanked, or Flanking Enemy Lord. A Player with a Flank-
ing Lord selects either the Flanking or directly opposed Lord to 
take Hits. Whenever a Lord Routs to create a new Flanking situa-
tion, apply remaining Hits accordingly. 
PROTECTION: Strongholds, Siegeworks, cards, and most 
Forces types give saving rolls that can nullify Hits—Walls, Ar-
mor, Evade, and Unarmored Protection. Crossbows and cards can 
alter the rolls needed. See the Forces and Strongholds tables.
ROLL WALLS: If this is a Storm or Sally (4.5.2-.3), a Lord ben-
efiting from Walls or Siegeworks first rolls dice equal to the total 
number of Hits just received in that step before assigning Hits to 
units. Each roll within the Walls range specified (often, 1-4; 1-3 
with Siege Towers) or less than or equal to the total Siege mark-
ers there cancels one Hit.
                """ },            
            {"role": "user", "content": "Please explain to me rules of Battle in Almoravid"}
        ]
    )

In [23]:
for c in answer.choices:
    print(c.message.content)

Certainly! The battle mechanics in Almoravid can be intricate, but I’ll break them down into key steps and principles for clarity.

### Overview of Battle

1. **Initiation**: Battles consist of rounds where players take turns striking at each other using their Lords (units). Each battle lasts at least one round.

2. **Pursuit Advantage**: If one side concedes, the enemy gains a "Pursuit" advantage, which affects the Hits applied to the conceding side.

### Key Steps in Battle

1. **Repositioning Lords**: At the beginning of each round after the first, both sides will reposition their Lords. The steps are:
   - **Rout**: Remove any Routed Lords from the battle array; these units cannot participate further until the end of the battle.
   - **Advance**: Move any Unrouted Lords from Reserve into empty front positions (one Lord per position).
   - **Center**: If there is still an empty center position in the front row, each side must slide one Lord from either side to occupy it.

2. **Strik

In [1]:
%pip install -qU rouge-score

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# https://thepythoncode.com/article/calculate-rouge-score-in-python
from rouge_score import rouge_scorer
import numpy as np

test_questions_and_answers = [
    ("What is the full name of the game?", "Almoravid: Reconquista and Riposte in Spain, 1085-1086"),
    ("When does the game take place?", "The game takes place during mediaval reconquista in Iberian peninsula, specifically in the years 1085-1086"),
    ("What do the opponents represent?", "The opponents represent the muslim Taifas (green color) and the Christian forces (yellow) led by Alphonso VI of Leon and Castile")
]

llm_answers = [	
    "The game is called Almoravid",
    "The game represents the mediaval period",
    "The opponents are yellow and green"
]

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge1_scores, rouge2_scores, rougeL_scores = [], [], []



for ref, gen in zip(test_questions_and_answers, llm_answers) :
    scores = scorer.score(ref[1], gen)
    print(scores)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)


In [23]:
for q,a in test_questions_and_answers:
    print(q)
    print(a)

What is the full name of the game?
Almoravid: Reconquista and Riposte in Spain, 1085-1086
When does the game take place?
The game takes place during mediaval reconquista in Iberian peninsula, specifically in the years 1085-1086
What do the opponents represent?
The opponents represent the muslim Taifas (green color) and the Christian forces (yellow) led by Alphonso VI of Leon and Castile


In [ ]:
for index, ref in enumerate(test_questions_and_answers):
    print("Reference:", ref)
    print("Generated:", llm_answers[index])
    print("Rouge1:", rouge1_scores[index])
    print("Rouge2:", rouge2_scores[index])
    print("RougeL:", rougeL_scores[index])

In [21]:
# Aggregate ROUGE scores
avg_rouge1 = np.mean(rouge1_scores)
avg_rouge2 = np.mean(rouge2_scores)
avg_rougeL = np.mean(rougeL_scores)

# Print the results
print(f"Average ROUGE-1 F1 Score: {avg_rouge1:.4f}")
print(f"Average ROUGE-2 F1 Score: {avg_rouge2:.4f}")
print(f"Average ROUGE-L F1 Score: {avg_rougeL:.4f}")

Average ROUGE-1 F1 Score: 0.2960
Average ROUGE-2 F1 Score: 0.0600
Average ROUGE-L F1 Score: 0.2410
